In [2]:
!pip install python-terrier

In [3]:
# Need to install additional terrier package for PRF. It will take around 1 min
!git clone https://github.com/terrierteam/terrier-prf/
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

Cloning into 'terrier-prf'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 227 (delta 62), reused 192 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (227/227), 33.94 KiB | 914.00 KiB/s, done.
Resolving deltas: 100% (62/62), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaopalliance-java libapache-pom-java libatinject-jsr330-api-java libcdi-api-java
  libcommons-cli-java libcommons-io-java libcommons-lang3-java libcommons-parent-java
  libgeronimo-annotation-1.3-spec-java libgeronimo-interceptor-3.0-spec-java libguava-java
  libguice-java libhawtjni-runtime-java libjansi-java libjansi-native-java libjsr305-java
  libmaven-parent-java libmaven-resolver-java libmaven-shared-utils-java libmaven3-core-java
  libplexus-cipher-java libplexus-classworlds-java libple

In [4]:
import pyterrier as pt
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

<ipython-input-4-44fbee2fda4f>:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done
terrier-assemblies 5.10 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
<ipython-input-4-44fbee2fda4f>:4: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
The following code will have the same effect:
pt.java.add_package('com.github.terrierteam', 'terrier-prf', '-SNAPSHOT')
pt.java.init() # optional, forces java initialisation
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])


In [5]:
from google.colab import drive
import pandas as pd
import numpy as np
import nltk
import pyterrier as pt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from nltk.stem import *
from nltk.stem.porter import *
from collections import Counter
import torch
from torch.nn.functional import softmax
pd.set_option('display.max_colwidth', 150)

In [6]:
# Libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

In [7]:
#Import the necessary modules:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

pd.set_option('display.max_colwidth', 150)
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
#  configures the Python environment for text processing tasks
pd.set_option('display.max_colwidth', 150)
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data Collection

In [9]:
# Vaswani Dataset
# Retrieves topics, relevance judgments, and corpus path from the Vaswani dataset.
vaswani_dataset = pt.datasets.get_dataset("vaswani")
topics = vaswani_dataset.get_topics()
qrels = vaswani_dataset.get_qrels()
data_path = vaswani_dataset.get_corpus()[0]

query-text.trec:   0%|          | 0.00/3.05k [00:00<?, ?iB/s]

qrels:   0%|          | 0.00/6.63k [00:00<?, ?iB/s]

doc-text.trec:   0%|          | 0.00/0.99M [00:00<?, ?iB/s]

In [10]:
# splits documents by '<DOCNO>', and stores them in a DataFrame; then sets up the 'documents' column and removes the initial empty column.
with open(data_path, 'r') as file:
    documents = file.read()

documents = documents.split('<DOCNO>')
df = pd.DataFrame(documents)
# the documents stored in the vaswani_dataset
df['documents'] = df
df.drop(0, axis = 1, inplace = True)

Preprocessing

In [11]:
def clean(text):
   text = re.sub(r"http\S+", " ", text) # remove urls
   text = re.sub(r"RT ", " ", text) # remove rt
   text = re.sub(r"@[\w]*", " ", text) # remove handles
   text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
   text = re.sub(r'\t', ' ', text) # remove tabs
   text = re.sub(r'\n', ' ', text) # remove line jump r'\d'
   text = re.sub(r"\s+", " ", text) # remove extra white space
   text = re.sub(r" < doc > < doc >", " ", text)
   text = re.sub(r"< docno > ", " ", text)
   text = text.strip()
   return text


In [12]:
def Steem_text(text):
    stemmer = PorterStemmer()
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    # print (tokens)
    return ' '.join(stemmed_tokens)

In [13]:
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words] #Lower is used to normalize al the words make them in lower case
    return ' '.join(filtered_tokens)

In [14]:
def preprocess(sentence):
  sentence = remove_stopwords(sentence)
  sentence = clean(sentence)
  sentence = Steem_text(sentence)
  return sentence

In [15]:
# Applies preprocessing to documents, removes digits from processed and original data.
df['processed_data'] = df['documents'].apply(preprocess)
df['processed_data']  = df['processed_data'].replace(to_replace=r'\d', value='', regex=True)  # this line removes any digit found in the text
df['documents']  = df['documents'].replace(to_replace=r'\d', value='', regex=True)

Indexing

In [16]:
# Creates a document number for each entry, converts it to a string, and applies text cleaning and stopword removal to the documents.
index = list()
for i in range(len(df)):
  index.append(i)
df['docno'] = index
df['docno'] = df['docno'].astype(str)
df['documents'] = df['documents'].apply(remove_stopwords).apply(clean)
df

,documents,processed_data,docno
0,< doc >,< doc >,0
1,compact memories flexible capacities digital data storage system capacity bits random sequential access described,compact memori flexibl capac digit data storag system capac bit random sequenti access describ,1
2,electronic analogue computer solving systems linear equations mathematical derivation operating principle stability conditions computer consisting...,electron analogu comput solv system linear equat mathemat deriv oper principl stabil condit comput consist amplifi,2
3,electronic coordinate transformer circuit details given construction electronic calculating unit enables polar coordinates vector modulus cosine s...,electron coordin transform circuit detail given construct electron calcul unit enabl polar coordin vector modulu cosin sine argument deriv rectan...,3
4,british computer society report conference held cambridge june,british comput societi report confer held cambridg june,4
...,...,...,...
11425,diurnal power variation earth ionosphere cavity modes relationship world wide thunderstorm activity,diurnal power variat earth ionospher caviti mode relationship world wide thunderstorm activ,11425
11426,design small economical radio frequency eht supplies particular attention given design high voltage transformer type stabilized eht generator desc...,design small econom radio frequenc eht suppli particular attent given design high voltag transform type stabil eht gener describ suitabl oper fre...,11426
11427,satellite power supply variable pulse width detailed circuit description short circuit proof dc dc converter high efficiency input voltage output ...,satellit power suppli variabl puls width detail circuit descript short circuit proof dc dc convert high effici input voltag output voltag held power,11427
11428,solar batteries use power source unattended uhf repeaters,solar batteri use power sourc unattend uhf repeat,11428


In [17]:
indexer = pt.DFIndexer("./DatasetIndex", overwrite=True)
# index the text, record the docnos as metadata
index_ref = indexer.index(df["processed_data"], df["docno"])
index_ref.toString()

<ipython-input-17-d50077ea0e77>:1: DeprecationWarning: Call to deprecated class DFIndexer. (use pt.terrier.IterDictIndexer().index(dataframe.to_dict(orient='records')) instead) -- Deprecated since version 0.11.0.
  indexer = pt.DFIndexer("./DatasetIndex", overwrite=True)


13:33:51.482 [main] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (0) - further warnings are suppressed
13:34:03.949 [main] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents


'./DatasetIndex/data.properties'

In [18]:
index = pt.IndexFactory.of(index_ref)

In [19]:
meta = index.getMetaIndex()
inv = index.getInvertedIndex()
lex = index.getLexicon()

In [20]:
# Maps unique terms (terms appearing in only one document) in the index to their respective document numbers.
unique_terms = {}
# mapping unique terms
for kv in index.getLexicon():
    term = kv.getKey()
    le = index.getLexicon().getLexiconEntry(term)
    docnos = [meta.getItem("docno", posting.getId()) for posting in inv.getPostings(le)]
    if le.getDocumentFrequency() == 1:
      unique_terms[term] = docnos

unique_terms

{'aachen': ['1537'],
 'abandon': ['6667'],
 'abd': ['5234'],
 'aber': ['3076'],
 'ablat': ['10420'],
 'abomb': ['118'],
 'abraham': ['3786'],
 'abridg': ['1537'],
 'acceleromet': ['4810'],
 'accentu': ['5650'],
 'accessori': ['7706'],
 'accid': ['4178'],
 'accomod': ['7069'],
 'accordingli': ['6709'],
 'accret': ['3001'],
 'accross': ['9737'],
 'acetaldehyd': ['6016'],
 'aceton': ['6016'],
 'achimota': ['7977'],
 'acknowledg': ['8637'],
 'acm': ['6420'],
 'acoustoelectr': ['2350'],
 'acut': ['8273'],
 'ada': ['3253'],
 'addam': ['2174'],
 'adjoint': ['9686'],
 'adjustabl': ['6166'],
 'administr': ['1537'],
 'admira': ['5424'],
 'admixtur': ['2364'],
 'advanect': ['1466'],
 'advisori': ['10953'],
 'advoc': ['7651'],
 'aee': ['8194'],
 'aeriel': ['5401'],
 'aerolen': ['6692'],
 'aerolog': ['2401'],
 'aeromagnet': ['6210'],
 'aeromagnetomet': ['5521'],
 'aeronom': ['6489'],
 'aeronomi': ['10554'],
 'aerosol': ['3546'],
 'afbmd': ['10563'],
 'afghanistan': ['7526'],
 'agassiz': ['5393'],
 

In [21]:
df

,documents,processed_data,docno
0,< doc >,< doc >,0
1,compact memories flexible capacities digital data storage system capacity bits random sequential access described,compact memori flexibl capac digit data storag system capac bit random sequenti access describ,1
2,electronic analogue computer solving systems linear equations mathematical derivation operating principle stability conditions computer consisting...,electron analogu comput solv system linear equat mathemat deriv oper principl stabil condit comput consist amplifi,2
3,electronic coordinate transformer circuit details given construction electronic calculating unit enables polar coordinates vector modulus cosine s...,electron coordin transform circuit detail given construct electron calcul unit enabl polar coordin vector modulu cosin sine argument deriv rectan...,3
4,british computer society report conference held cambridge june,british comput societi report confer held cambridg june,4
...,...,...,...
11425,diurnal power variation earth ionosphere cavity modes relationship world wide thunderstorm activity,diurnal power variat earth ionospher caviti mode relationship world wide thunderstorm activ,11425
11426,design small economical radio frequency eht supplies particular attention given design high voltage transformer type stabilized eht generator desc...,design small econom radio frequenc eht suppli particular attent given design high voltag transform type stabil eht gener describ suitabl oper fre...,11426
11427,satellite power supply variable pulse width detailed circuit description short circuit proof dc dc converter high efficiency input voltage output ...,satellit power suppli variabl puls width detail circuit descript short circuit proof dc dc convert high effici input voltag output voltag held power,11427
11428,solar batteries use power source unattended uhf repeaters,solar batteri use power sourc unattend uhf repeat,11428


In [22]:
# Constructs a term frequency dictionary for each document, counting occurrences of each term.
def tf(processed_docs):
    tf_dict = {}
    for doc_id, doc_text in enumerate(processed_docs):
        doc_tokens = doc_text.split()
        for term in doc_tokens:
            if term not in tf_dict:
                tf_dict[term] = {}
            if doc_id not in tf_dict[term]:
                tf_dict[term][doc_id] = 0
            tf_dict[term][doc_id] += 1
    return tf_dict

processed_docs = df['processed_data'].tolist()
tf_dict = tf(processed_docs)
print(tf_dict)

{'<': {0: 1, 11429: 1}, 'doc': {0: 1, 11429: 1}, '>': {0: 1, 11429: 1}, 'compact': {1: 1, 510: 1, 2590: 1, 3304: 1, 3363: 1, 3395: 1, 3593: 1, 5631: 1, 6547: 1, 6800: 1, 7779: 1, 9304: 1, 9544: 1}, 'memori': {1: 1, 512: 1, 514: 1, 638: 1, 773: 1, 923: 1, 1158: 1, 1159: 1, 1160: 1, 1173: 1, 1195: 1, 1298: 1, 1309: 1, 1593: 1, 1597: 1, 1712: 1, 1837: 1, 2177: 1, 2181: 1, 2291: 2, 2424: 1, 2679: 1, 2790: 1, 2930: 1, 3047: 1, 3141: 1, 3142: 1, 3147: 1, 3162: 1, 3250: 1, 3254: 1, 3261: 1, 3377: 5, 3378: 1, 3945: 1, 3953: 1, 3954: 1, 4061: 1, 4572: 1, 4931: 1, 5434: 1, 5446: 1, 5447: 1, 5452: 1, 5458: 1, 5653: 1, 5668: 1, 5836: 1, 5837: 1, 5956: 2, 6048: 1, 6235: 1, 7758: 1, 7990: 1, 7991: 1, 8066: 1, 8527: 1, 8647: 1, 8812: 1, 9070: 1, 9077: 1, 9277: 1, 9375: 1, 9403: 1, 10157: 1, 10471: 1, 10474: 1, 10615: 1, 10736: 1, 10737: 1, 10873: 1, 10876: 2, 11021: 1, 11043: 1, 11170: 1, 11293: 1}, 'flexibl': {1: 1, 4524: 1, 4615: 1, 5682: 1, 6340: 1, 6905: 1, 7273: 1, 8124: 1, 9164: 2, 9675: 1}, 'c

In [23]:
# Prints each term from the index lexicon with its corresponding lexicon entry details.
for kv in index.getLexicon():
  print("%s -> %s " % (kv.getKey(), kv.getValue().toString().split('@')[0]))

Streaming output truncated to the last 5000 lines.
gossard -> term2734 Nt=1 TF=1 maxTF=1  
gottingen -> term6289 Nt=1 TF=1 maxTF=1  
gould -> term7227 Nt=1 TF=1 maxTF=1  
govenr -> term6593 Nt=1 TF=1 maxTF=1  
govern -> term351 Nt=28 TF=28 maxTF=1  
gpa -> term5971 Nt=1 TF=1 maxTF=1  
graaff -> term6667 Nt=2 TF=2 maxTF=1  
grad -> term6628 Nt=1 TF=1 maxTF=1  
grade -> term374 Nt=9 TF=9 maxTF=1  
gradient -> term1190 Nt=67 TF=82 maxTF=3  
gradual -> term2250 Nt=12 TF=12 maxTF=1  
graduat -> term4614 Nt=3 TF=3 maxTF=1  
graffund -> term6760 Nt=1 TF=1 maxTF=1  
grahamstown -> term1657 Nt=2 TF=2 maxTF=1  
grai -> term1702 Nt=2 TF=2 maxTF=1  
grain -> term3355 Nt=6 TF=7 maxTF=2  
gram -> term5188 Nt=2 TF=3 maxTF=2  
gramophon -> term4959 Nt=4 TF=6 maxTF=3  
grand -> term7630 Nt=1 TF=1 maxTF=1  
granit -> term3926 Nt=1 TF=1 maxTF=1  
granular -> term4677 Nt=2 TF=2 maxTF=1  
graph -> term731 Nt=77 TF=89 maxTF=3  
graphic -> term1494 Nt=191 TF=200 maxTF=2  
graphit -> term4464 Nt=3 TF=3 maxTF=

Query Processing

In [24]:
# Matches documents that contain all tokens from a query and prints the matched document texts.
def match_docs(query, df):
  tokens = query.split()
  res = list()
  for id in df['docno']:
    counter = 0
    for token in tokens:
      if token in str(df[df['docno']== str(id)]['processed_data']):
        counter = counter + 1
      if counter == len(tokens):
        res.append(id)
  for idx in range(len(res)):
    print(f'Text:', str(df[df['docno']== str(res[idx])]['documents']))

  return res

In [25]:
Query= 'diurnal power variat earth'
Query= preprocess(Query)

In [26]:
tfidf_retr = pt.BatchRetrieve(index,wmodel="TF_IDF",num_results=10)
results = tfidf_retr.search(Query)
print(results)

<ipython-input-26-0f79decddb3d>:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf_retr = pt.BatchRetrieve(index,wmodel="TF_IDF",num_results=10)


  qid  docid  docno  rank      score                       query
0   1  11425  11425     0  12.686517  diurnal power variat earth
1   1   2122   2122     1   9.996584  diurnal power variat earth
2   1   1551   1551     2   9.981959  diurnal power variat earth
3   1   8080   8080     3   9.808489  diurnal power variat earth
4   1   6212   6212     4   9.767483  diurnal power variat earth
5   1    337    337     5   7.944752  diurnal power variat earth
6   1  11270  11270     6   7.871474  diurnal power variat earth
7   1    125    125     7   7.696403  diurnal power variat earth
8   1   7426   7426     8   7.615126  diurnal power variat earth
9   1   1421   1421     9   7.567100  diurnal power variat earth


In [27]:
results2 = match_docs(Query, df[11424: ])
print(results2)

Text: 11425    diurnal power variation earth ionosphere cavity modes relationship world wide thunderstorm activity
Name: documents, dtype: object
['11425']


In [28]:
# Retrieves and prints document IDs and frequency of preprocessed terms from the index.
meta = index.getMetaIndex()
inv = index.getInvertedIndex()
lex = index.getLexicon()

# List of terms to retrieve document frequency
terms = preprocess("british comput societi report")  # Add your terms here

for term in terms.split():
    le = lex.getLexiconEntry(term)
    if le is None:
        print(f"Term '{term}' not found in the lexicon.")
        continue

    # Access the inverted index posting list for the term
    for posting in inv.getPostings(le):
        docno = meta.getItem("docno", posting.getId())
        print(f"Document: {docno}, Frequency: {posting.getFrequency()} for term: {term}")

Document: 4, Frequency: 1 for term: british
Document: 408, Frequency: 1 for term: british
Document: 497, Frequency: 1 for term: british
Document: 1592, Frequency: 1 for term: british
Document: 1845, Frequency: 1 for term: british
Document: 2005, Frequency: 1 for term: british
Document: 2028, Frequency: 2 for term: british
Document: 3601, Frequency: 1 for term: british
Document: 3812, Frequency: 1 for term: british
Document: 4146, Frequency: 1 for term: british
Document: 4589, Frequency: 1 for term: british
Document: 5819, Frequency: 1 for term: british
Document: 6827, Frequency: 1 for term: british
Document: 8938, Frequency: 1 for term: british
Document: 10141, Frequency: 2 for term: british
Document: 10254, Frequency: 1 for term: british
Document: 2, Frequency: 2 for term: comput
Document: 4, Frequency: 1 for term: comput
Document: 5, Frequency: 1 for term: comput
Document: 69, Frequency: 1 for term: comput
Document: 140, Frequency: 4 for term: comput
Document: 141, Frequency: 3 for t

Query Expansion

In [29]:
query = preprocess("solar")
TF_IDF = pt.BatchRetrieve(index, wmodel="TF_IDF",num_results=10)
resultss = TF_IDF.search(query)
resultss

<ipython-input-29-edd284184018>:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  TF_IDF = pt.BatchRetrieve(index, wmodel="TF_IDF",num_results=10)


,qid,docid,docno,rank,score,query
0,1,9018,9018,0,3.997293,solar
1,1,10541,10541,1,3.944266,solar
2,1,8401,8401,2,3.915317,solar
3,1,10265,10265,3,3.804438,solar
4,1,11390,11390,4,3.804438,solar
5,1,6020,6020,5,3.785916,solar
6,1,3805,3805,6,3.750173,solar
7,1,11147,11147,7,3.723994,solar
8,1,10551,10551,8,3.715599,solar
9,1,77,77,9,3.687804,solar


In [30]:
query = preprocess("electronic")
bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=10)
results = bm25.search(query)
results

<ipython-input-30-319d68fb592a>:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=10)


,qid,docid,docno,rank,score,query
0,1,9120,9120,0,4.765505,electron
1,1,2725,2725,1,4.686740,electron
2,1,9449,9449,2,4.659349,electron
3,1,8867,8867,3,4.568595,electron
4,1,4854,4854,4,4.550355,electron
5,1,10723,10723,5,4.550355,electron
6,1,5896,5896,6,4.525164,electron
7,1,9448,9448,7,4.524531,electron
8,1,4464,4464,8,4.524215,electron
9,1,1766,1766,9,4.524026,electron


In [31]:
rm3_expander = pt.rewrite.RM3(index, fb_terms=10, fb_docs=100)
rm3_qe = bm25 >> rm3_expander
expanded_query = rm3_qe.search(query).iloc[0]["query"]
expanded_query

'applypipeline:off secondari^0.038256858 descript^0.042455148 contribut^0.031616621 ga^0.038103756 energi^0.044640563 metal^0.037423715 collect^0.029943235 interact^0.080809526 accel^0.029723618 scatter^0.027026905 electron^0.600000024'

In [32]:
print(expanded_query)

applypipeline:off secondari^0.038256858 descript^0.042455148 contribut^0.031616621 ga^0.038103756 energi^0.044640563 metal^0.037423715 collect^0.029943235 interact^0.080809526 accel^0.029723618 scatter^0.027026905 electron^0.600000024


In [33]:
# Just print the expanded query with term scores
for s in expanded_query.split()[1:]:
  print(s)
print("\n" + query)

secondari^0.038256858
descript^0.042455148
contribut^0.031616621
ga^0.038103756
energi^0.044640563
metal^0.037423715
collect^0.029943235
interact^0.080809526
accel^0.029723618
scatter^0.027026905
electron^0.600000024

electron


In [34]:
results

,qid,docid,docno,rank,score,query
0,1,9120,9120,0,4.765505,electron
1,1,2725,2725,1,4.686740,electron
2,1,9449,9449,2,4.659349,electron
3,1,8867,8867,3,4.568595,electron
4,1,4854,4854,4,4.550355,electron
5,1,10723,10723,5,4.550355,electron
6,1,5896,5896,6,4.525164,electron
7,1,9448,9448,7,4.524531,electron
8,1,4464,4464,8,4.524215,electron
9,1,1766,1766,9,4.524026,electron


In [35]:
# After that you can search using the expanded query
expanded_query_formatted = ' '.join(expanded_query.split()[1:])
results_wqe = bm25.search(expanded_query_formatted)
print("   Before Expansion    After Expansion")
print(pd.concat([results[['docid','score']][0:5].add_suffix('_1'),
            results_wqe[['docid','score']][0:5].add_suffix('_2')], axis=1).fillna(''))
#Let's check the tweets text for the top 5 retrieved tweets
df[['processed_data']][df['docno'].isin(results_wqe['docno'].loc[0:5].tolist())]

   Before Expansion    After Expansion
   docid_1   score_1  docid_2   score_2
0     9120  4.765505     5896  7.639768
1     2725  4.686740     4464  7.216497
2     9449  4.659349     5895  7.163942
3     8867  4.568595     5784  6.929092
4     4854  4.550355     1225  6.869256


,processed_data
1225,ferromagnet electron ga studi made basi bohm pine collect descript electron interact
4464,electron phonon interact metal role electron electron interact determin th electron phonon interact metal investig extend bohm pine collect descr...
5784,modifi theori product secondari electron solid difficulti previou theori energi loss secondari electron product metal remov replac coulomb intera...
5895,collect descript electron interact part coulomb interact degener electron ga part april
5896,collect descript electron interact part electron interact metal
9120,spin suscept electron ga gener random phase approxim use investig effect electron electron interact spin suscept electron ga


Embeddings (ELMo)

In [36]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

In [37]:
# Function to calculate cosine similarity
def cosine_similarity(v1, v2):
    # Ensure the embeddings are summed across all tokens and reduced to a 1D array
    sum_v1 = np.sum(v1, axis=0)
    sum_v2 = np.sum(v2, axis=0)
    if sum_v1.ndim > 1:
        sum_v1 = np.sum(sum_v1, axis=0)
    if sum_v2.ndim > 1:
        sum_v2 = np.sum(sum_v2, axis=0)

    dot_product = np.dot(sum_v1, sum_v2)
    norm_v1 = np.linalg.norm(sum_v1)
    norm_v2 = np.linalg.norm(sum_v2)
    return dot_product / (norm_v1 * norm_v2)

In [38]:
# Load the ELMo model
elmo = hub.load("https://tfhub.dev/google/elmo/3")

# Define the query sentence
query_sentence = "solar"

# Assuming docs_df is your DataFrame with a 'processed_text' column
# Limit the loop to the first 10 rows
similarities = []
for doc_content in df['processed_data'][df['docno'].isin(results['docno'].loc[0:9].tolist())]:  # Process only the first 10 documents
    # Generate ELMo embeddings for the query sentence and the document content
    query_embedding = elmo.signatures["default"](tf.constant([query_sentence]))["elmo"].numpy()
    doc_embedding = elmo.signatures["default"](tf.constant([doc_content]))["elmo"].numpy()

    # Calculate cosine similarity between the query and the document
    similarity_score = cosine_similarity(query_embedding, doc_embedding)
    similarities.append(similarity_score)

In [39]:
# Create a DataFrame to store similarity scores with corresponding documents
similarity_df = pd.DataFrame({
    'Document': df['processed_data'][df['docno'].isin(results['docno'].loc[0:9].tolist())],  # Include only the first 10 documents
    'Similarity Score': similarities
})

# Rank the documents based on similarity scores
similarity_df = similarity_df.sort_values(by='Similarity Score', ascending=False)

# Print the ranked DataFrame
print(similarity_df)

                                                                                                                                                    Document  \
10723   transreson electron acceler describ mechan addit helliwel acceler van allen electron whistler non relativist electron would acceler whistler fiel...   
2725    electron emiss surfac pure molybdenum bombard electron low energi electron bombard period sec pure mo surfac room temperatur dark found emit elec...   
4464    electron phonon interact metal role electron electron interact determin th electron phonon interact metal investig extend bohm pine collect descr...   
8867                                 measur electron densiti plasma evolut experiment apparatu gener descript apparatu measur electron densiti rang electron   
1766    transport phenomena complet ioniz ga consid electron electron scatter hall mobil transport properti electron complet ioniz ga investig magnet fie...   
5896                                    

User Interface

In [40]:
!pip install flask pyngrok

In [41]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok

In [ ]:
# df['processed_data'] = df['documents'].apply(preprocess)
# df['processed_data']
# df['docno']=df['docno'].apply(str)

# indexer = pt.DFIndexer("./myFirstIndex2", overwrite=True)

# index_ref = indexer.index(df["processed_data"], df["docno"],df["documents"])
# print(index_ref.toString())
# index = pt.IndexFactory.of(index_ref)
# print(index.getCollectionStatistics().toString())
# tokens2 =  df['processed_data'].apply(word_tokenize)
# ngrok.set_auth_token('2gI5T14n4zi00vTgYeNlW6AsIRC_4DUGgYFTdCxqMuj3UZVD')
# Import the necessary libraries
# Flask App Initialization
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import pandas as pd
import pyterrier as pt
from nltk.tokenize import word_tokenize

# Flask App Initialization
app = Flask(__name__)

# Start ngrok tunnel (replace with your actual auth token)
ngrok.set_auth_token('2gI5T14n4zi00vTgYeNlW6AsIRC_4DUGgYFTdCxqMuj3UZVD')
public_url = ngrok.connect(5000)
print('NGROK Tunnel URL:', public_url)

# Preprocessing function
def preprocess(text):
    return text.lower().strip()

# HTML Template for displaying the document
DOCUMENT_TEMPLATE = '''
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Document {{ docno }}</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      margin: 0;
      padding: 0;
      background-color: #f4f4f4;
      color: #333;
      display: flex;
      justify-content: center;
      align-items: center;
      height: 100vh;
    }
    .container {
      text-align: center;
      background-color: #fff;
      padding: 40px;
      border-radius: 10px;
      box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
      width: 50%;
    }
    h1 {
      color: #4CAF50;
      font-size: 28px;
      margin-bottom: 20px;
    }
    h2 {
      font-size: 22px;
      font-weight: bold;
      margin-bottom: 20px;
    }
    p {
      font-size: 18px;
      text-align: left;
      line-height: 1.6;
    }
  </style>
</head>
<body>
  <div class="container">
    <h1>Welcome to Infotex Engine</h1>
    <h2>Document Number: <strong>{{ docno }}</strong></h2>
    <p>{{ content }}</p>
  </div>
</body>
</html>
'''

# HTML Template for the home and search results page
HTML_TEMPLATE = '''
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Infotex Engine</title>
  <style>
    body {
      font-family: Arial, sans-serif;
      margin: 0;
      padding: 0;
      text-align: center;
      background-color: #000;
      color: #fff;
    }
    header {
      background-color: #222;
      color: #fff;
      padding: 20px 0;
      border-bottom: 1px solid #444;
      position: fixed;
      width: 100%;
      top: 0;
      left: 0;
      right: 0;
      z-index: 100;
    }
    form {
      margin: 100px auto 20px;
      padding: 20px;
      width: 40%;
    }
    input[type="text"] {
      width: 70%;
      height: 44px;
      padding: 10px 20px;
      font-size: 16px;
      border: 1px solid #555;
      background-color: #333;
      color: #ddd;
      margin-bottom: 20px;
      border-radius: 22px;
    }
    input[type="submit"] {
      height: 44px;
      width: 20%;
      background-color: #4CAF50;
      font-size: 16px;
      border: none;
      border-radius: 22px;
      cursor: pointer;
    }
    ul {
      list-style: none;
      padding: 0;
      margin-top: 20px;
      text-align: left;
      position: relative;
    }
    li {
      background-color: #222;
      color: #fff;
      margin: 5px auto;
      padding: 20px;
      border: 1px solid #444;
      width: 40%;
      box-shadow: 0px 2px 5px rgba(0,0,0,0.5);
    }
    a {
      text-decoration: none;
      color: #9DFF9D;
      font-size: 18px;
    }
    a:hover {
      text-decoration: underline;
    }
  </style>
</head>
<body>
  <header>
    <h1>Infotex Engine</h1>
    <form action="/search" method="post">
      <input type="text" name="query" placeholder="Enter search terms...">
      <input type="submit" value="Search">
    </form>
  </header>
  <ul>
    {% if query %}
      <h2 style="margin: 160px 0 20px; color: #777;">Results for: "{{ query }}"</h2>
      {% if results %}
        {% for doc, link, text in results %}
          <li><a href="{{ link }}">{{ text }} (Document: {{ doc }})</a></li>
        {% endfor %}
      {% else %}
        <p>No results found.</p>
      {% endif %}
    {% endif %}
  </ul>
</body>
</html>
'''

@app.route("/", methods=["GET"])
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route("/search", methods=["POST"])
def search():
    query = request.form['query']
    search_results = []

    # Assuming an index is created and available
    tfidf_retriever = pt.BatchRetrieve(index, controls={"wmodel": "BM25"}, num_results=30)
    query = preprocess(query)  # Ensure the query is preprocessed
    tfidf_results = tfidf_retriever.transform(query)

    # Handle results
    for idx, row in tfidf_results.iterrows():
        docno = row['docno']
        text = df[df['docno'] == docno]['documents'].values[0]
        link = f"/doc/{docno}"  # Link to the internal document route
        search_results.append((docno, link, text))

    return render_template_string(HTML_TEMPLATE, query=query, results=search_results)

# Route to display document
@app.route("/doc/<docno>")
def display_document(docno):
    content = df[df['docno'] == docno]['documents'].values[0]
    return render_template_string(DOCUMENT_TEMPLATE, docno=docno, content=content)

# Running the app
if __name__ == "__main__":
    app.run(port=5000)


NGROK Tunnel URL: NgrokTunnel: "https://4628-104-199-210-105.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Sep/2024 14:09:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Sep/2024 14:09:56] "GET /favicon.ico HTTP/1.1" 404 -
<ipython-input-67-3aefc4dc6866>:200: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf_retriever = pt.BatchRetrieve(index, controls={"wmodel": "BM25"}, num_results=30)
/usr/local/lib/python3.10/dist-packages/pyterrier/utils.py:207: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  return fn(*args, **kwargs)
INFO:werkzeug:127.0.0.1 - - [18/Sep/2024 14:10:01] "POST /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Sep/2024 14:10:04] "GET /doc/9449 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1

Evaluation

In [43]:
# Build retrieval model using BM25 which retrieves 15 results
bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=15)
retr1=bm25.transform(topics)
retr1

<ipython-input-43-ba54870fff45>:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=15)


,qid,docid,docno,rank,score,query
0,1,8172,8172,0,24.614757,measurement of dielectric constant of liquids by the use of microwave techniques
1,1,9881,9881,1,22.140010,measurement of dielectric constant of liquids by the use of microwave techniques
2,1,5502,5502,2,21.764363,measurement of dielectric constant of liquids by the use of microwave techniques
3,1,1502,1502,3,19.497202,measurement of dielectric constant of liquids by the use of microwave techniques
4,1,4871,4871,4,18.341602,measurement of dielectric constant of liquids by the use of microwave techniques
...,...,...,...,...,...,...
1390,93,10469,10469,10,20.172581,high frequency oscillators using transistors theoretical treatment and practical circuit details
1391,93,3860,3860,11,20.152545,high frequency oscillators using transistors theoretical treatment and practical circuit details
1392,93,3381,3381,12,20.043955,high frequency oscillators using transistors theoretical treatment and practical circuit details
1393,93,3537,3537,13,19.665067,high frequency oscillators using transistors theoretical treatment and practical circuit details


In [44]:
x=pt.Evaluate(retr1,qrels)
x

{'map': 0.16736711795501544, 'ndcg': 0.30322852249135596}

In [45]:
# Build retrieval model using TF_IDF which retrieves 15 results
TF_IDF = pt.BatchRetrieve(index, wmodel="TF_IDF",num_results=15)
retr2=TF_IDF.transform(topics)
retr2

<ipython-input-45-cce035950b8c>:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  TF_IDF = pt.BatchRetrieve(index, wmodel="TF_IDF",num_results=15)


,qid,docid,docno,rank,score,query
0,1,8172,8172,0,13.773330,measurement of dielectric constant of liquids by the use of microwave techniques
1,1,9881,9881,1,12.369113,measurement of dielectric constant of liquids by the use of microwave techniques
2,1,5502,5502,2,12.204707,measurement of dielectric constant of liquids by the use of microwave techniques
3,1,1502,1502,3,11.004179,measurement of dielectric constant of liquids by the use of microwave techniques
4,1,4871,4871,4,10.101840,measurement of dielectric constant of liquids by the use of microwave techniques
...,...,...,...,...,...,...
1390,93,7999,7999,10,11.647605,high frequency oscillators using transistors theoretical treatment and practical circuit details
1391,93,2696,2696,11,11.632124,high frequency oscillators using transistors theoretical treatment and practical circuit details
1392,93,8459,8459,12,11.584147,high frequency oscillators using transistors theoretical treatment and practical circuit details
1393,93,8468,8468,13,11.536246,high frequency oscillators using transistors theoretical treatment and practical circuit details


In [46]:
y=pt.Evaluate(retr2,qrels)
y

{'map': 0.1689370437355095, 'ndcg': 0.30410829945999246}

In [47]:
pt.Experiment(
    [TF_IDF, bm25],
    vaswani_dataset.get_topics(),
    vaswani_dataset.get_qrels(),
    eval_metrics=["map", "recip_rank","recall","P"],
    names=["TF_IDF", "BM25"]
)

,name,map,recip_rank,R@5,R@10,R@15,R@20,R@30,R@100,R@200,...,R@1000,P@5,P@10,P@15,P@20,P@30,P@100,P@200,P@500,P@1000
0,TF_IDF,0.168937,0.667730,0.146199,0.205243,0.252136,0.252136,0.252136,0.252136,0.252136,...,0.252136,0.440860,0.348387,0.297491,0.223118,0.148746,0.044624,0.022312,0.008925,0.004462
1,BM25,0.167367,0.673405,0.146333,0.206074,0.249677,0.249677,0.249677,0.249677,0.249677,...,0.249677,0.436559,0.344086,0.298208,0.223656,0.149104,0.044731,0.022366,0.008946,0.004473


In [48]:
topics

,qid,query
0,1,measurement of dielectric constant of liquids by the use of microwave techniques
1,2,mathematical analysis and design details of waveguide fed microwave radiations
2,3,use of digital computers in the design of band pass filters having given phase and attenuation characteristics
3,4,systems of data coding for information transfer
4,5,use of programs in engineering testing of computers
...,...,...
88,89,tunnel diode construction and its electrical characteristics explained
89,90,electronic density of states at the surface of a semiconductor compared with that at depth
90,91,resistivity of metallic thin films related to surface roughness
91,92,the phenomenon of radiation caused by charged particles moving in varying electric and magnetic fields


In [49]:
Query222=preprocess('measurement of dielectric ')
retr3=bm25.search(Query222)
retr3

,qid,docid,docno,rank,score,query
0,1,1502,1502,0,12.466975,measurement of dielectric
1,1,5145,5145,1,12.166863,measurement of dielectric
2,1,9881,9881,2,11.607994,measurement of dielectric
3,1,5502,5502,3,11.213095,measurement of dielectric
4,1,4936,4936,4,10.517502,measurement of dielectric
5,1,11033,11033,5,10.412306,measurement of dielectric
6,1,7735,7735,6,10.359606,measurement of dielectric
7,1,7923,7923,7,10.296640,measurement of dielectric
8,1,5031,5031,8,9.820865,measurement of dielectric
9,1,6471,6471,9,9.780139,measurement of dielectric


In [50]:
z=pt.Evaluate(retr3,qrels)
z

{'map': 0.0014761365780041503, 'ndcg': 0.0036068633221432813}